<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C11-Offensive_Forensics/Installation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Offensive Forensics__*

Forensics folks are often the people called in after a breach, or to determine if an "incident" has taken place at all. They typically want a snapshot of the affected machine's RAM in order to capture cryptographic keys or other information that resides only in memory. Lucky for them, a team of talented developers has created an entire Python framework called __Volatility__ that's suitable for this task and is billed as an advanced memory forensics framework. Incident responders, forensic examiners, and malware analysts can use Volatility for a variety of other tasks as well, including inspecting kernel objects, examining and dumping processes, and so on.

Although Volatility is software for the defensive side, any sufficiently powerful tool can be used for offense or defense. We will use Volatility to perform reconnaissance on a target user and write our own offensive plug-ins to search for weakly defended processes running on a virtual machine (__VM__). Suppose you infiltrate a machine and discover that the user employs a VM for sensitive work. Chances are good that the user has also made a snapshot of the VM as a safety net in case anything goes wrong with it. We will use the Volatility memory analysis framework to analyze the snapshot to find out how the VM is used and what processes were running. We'll also investigate possible vulnerabilities we can leverage for further exploitation. Let's get started!

### *__Installation__*

Volatility has been around for several years and has just undergone a complete rewrite. Not only is the code base now founded on Python 3, but the entire framework has been refactored so that the component are independent; all state required to run a plug-in is self-contained.

Let's create a virtual environment just for our work with Volatility. For this example, we are using Python 3 on a Widows machine in a PowerShell terminal. If you are also working from a Windows machine, make sure you have __git__ installed. You can download it at https://git-scm.com/downloads/.

```
PS> python3 -m venv vol3 #[1]
PS> vol3/Scripts/Activate.ps1
PS> cd vol3/
PS> git clone https//github.com/volatilityfoundation/volatility3.git #[2]
PS> cd volatility3/
PS> python setup.py install
PS> pip install pycryptodome #[3]
```

First, we create a new virtual environment called __vol3__ and activate it __[1]__. Next, we move into the virtual environment directory and clone the Volatility 3 GitHub repo __[2]__, install it into the virtual environment, and finally install __pycryptodome__ __[3]__, which we'll need later.

To see the plug-ins Volatility offers, as well as a list of options, use the following command on Windows:

```
PS> vol --help
```

On Linux or Mac, use the Python executable from the virtual environment, as follows:

```
$> python vol.py --help
```

In this chapter, we'll use Volatility from the command line, but there are various ways you might encounter the framework. For example, see the Volumetric project from Volatility, a free web-based GUI for volatility (https://github.com/volatilityfoundation/volumetric). You can dig into code examples in the Volumetric project to see how you can use Volatility in your own programs. Additionally, you can use the __volshell__ interface, which provides you with access to the Volatility framework and works as a normal interactive Python shell.

In the examples that follow, we'll use the Volatility command line. To save space, the output has been edited to show only the output discussed, so be aware that your output will have more lines and columns.

Now let's delve into some code and have a look inside the framework:

```
PS> cd volatility/framework/plugins/windows/
PS> ls
_init__.py    driverscan.py  memmap.py       psscan.py       vadinfo.py
bigpools.py   filescan.py    modscan.py      pstree.py       vadyarascan.py
cachedump.py  handles.py     modules.py      registry/       verinfo.py
callbacks.py  hashdump.py    mutantscan.py   ssdt.py         virtmap.py
cmdline.py    info.py        netscan.py      strings.py
dlllist.py    lsadump.py     poolscanner.py  svcscan.py
driverirp.py  malfind.py     pslist.py       symlinkscan.py  asdo
```

This listing shows the Python files inside the Windows __plugin__ directory. We highly encourage you to spend some time looking at the code in these files. You'll see a recurring pattern that forms the structure of a Volatility plug-in. This will help you understand the framework, but more importantly, it will give you a picture of a defender's mindset and intentions. By knowing what defenders are capable of and how they accomplish their objectives, you will make yourself into a more capable hacker and better understand how to protect yourself from detection.

Now that we have the analysis framework ready, we need some memory images to analyze. The easiest way to get one is to take a snapshot of your own Windows 10 virtual machine.

First, power up your Windows VM and start a few processes (for instance, the notepad, the calculator, and a browser); we'll examine the memory and track how these processes started. Then, take your snapshot using your hypervisor of choice. In the directory where your hypervisor stores your VMs, you'll see your new snapshot file with a name that ends with __.vmem__ or __.mem__. Let's start doing some recon!

Note that you can also find many memory images online. One image we'll look at in this chapter is provided by PassMark Software at https://www.osforensics.com/tools/volatility-workbench.html/. The Volatility Foundation site also has several images to play with at https://github.com/volatilityfoundation/volatility/wiki/Memory-Samples/.